In [62]:
# Practical 2

%matplotlib inline

import torch
from torch import Tensor
import dlc_practical_prologue as prologue
from matplotlib import pyplot as plt

In [4]:
# Load MNIST dataset (reduced set)
train_input, train_target, test_input, test_target = prologue.load_data(
    cifar = None, one_hot_labels = False, normalize = False, flatten = True)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [10]:
# Convert arrays to tensor and check their dimensions
train_input = torch.tensor(train_input)
print('train_input size: ',train_input.size())

train_target = torch.tensor(train_target)
print('train_target size: ',train_target.size())

test_input = torch.tensor(test_input)
print('test_input size: ',test_input.size())

test_target = torch.tensor(test_target)
print('test_target size: ',test_target.size())

train_input size:  torch.Size([1000, 784])
train_target size:  torch.Size([1000])
test_input size:  torch.Size([1000, 784])
test_target size:  torch.Size([1000])


c:\users\lucas waelti\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
c:\users\lucas waelti\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
c:\users\lucas waelti\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
c:\users\lucas waelti\appdata\local\programs\python\py

In [83]:
# Load CIFAR dataset (reduced set)
c_train_input, c_train_target, c_test_input, c_test_target = prologue.load_data(
    cifar = True, one_hot_labels = False, normalize = False, flatten = True)

* Using CIFAR
Files already downloaded and verified
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [84]:
# Convert arrays to tensor and check their dimensions
c_train_input = torch.tensor(c_train_input)
print('train_input size: ',c_train_input.size())

c_train_target = torch.tensor(c_train_target)
print('train_target size: ',c_train_target.size())

c_test_input = torch.tensor(c_test_input)
print('test_input size: ',c_test_input.size())

c_test_target = torch.tensor(c_test_target)
print('test_target size: ',c_test_target.size())

c:\users\lucas waelti\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
c:\users\lucas waelti\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
c:\users\lucas waelti\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


train_input size:  torch.Size([1000, 3072])
train_target size:  torch.Size([1000])
test_input size:  torch.Size([1000, 3072])
test_target size:  torch.Size([1000])


c:\users\lucas waelti\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


In [65]:
'''
    Exo 1) Nearest neighbours (k = 1)
    Compute the label of a datapoint x, based on its nearest neighbour
    train_input (n x d)
    train_target (n)
    x (1 X d) -> predict its label!
'''
def nearest_classification(train_input, train_target, x):
    diff = train_input - x
    diff2 = torch.pow(diff,2)
    _sum = torch.sum(diff2,1)
    
    
    # torch.sort(input, dim=None, descending=False, out=None) -> (sorted, indices)
    _,indices = torch.sort(_sum, dim=0)
    
    return train_target[indices[0]]

# Test the function with a chosen test sample
id = 100
print("Predicted label: ",nearest_classification(train_input,train_target,test_input[id,:]))
print("True label: ",test_target[id])
#plt.imshow(test_input[id,:])
    

Predicted label:  tensor(6)
True label:  tensor(6)


In [77]:
'''
    Exo 2) Error estimation
    Compute the predition for each datapoint of test_input. Count the number
    of missclassified points and return it. 
    Demean both train_input and train_target. Apply the transformation
    provided in proj (if not None). 
'''
def compute_nb_errors(train_input, train_target, test_input, test_target,
                        mean = None, proj = None):
    if(mean is not None):
        # Remove the provided mean 
        train_input = train_input - mean
        test_input = test_input - mean
        
    if(proj is not None):
        # Implement the provided projection
        train_input = proj.mm(train_input.t()).t()
        test_input = proj.mm(test_input.t()).t()
        
    # Classify the train and test inputs
    nb_errors = 0
    for x in range(0,test_input.size()[0]):
        prediction = nearest_classification(train_input,train_target,test_input[x,:])
        if(prediction.item() is not test_target[x].item()):
            nb_errors = nb_errors + 1
    
    return nb_errors

# Test the function 
print('Number of prediction errors: ',
      compute_nb_errors(train_input.clone(), train_target.clone(), 
                        test_input.clone(), test_target.clone()))

Number of prediction errors:  172


In [81]:
'''
    Exo 3) PCA
    Compute the PCA of the dataset X (n x d). return the mean (1 x d) and the basis of the PCA
'''
def PCA(X):
    # Demean the dataset
    mean = torch.mean(X, dim=0)
    X = X - mean
    # Compute the covariance matrix C = 1/M*X'*X
    M = X.size()[0]
    C = (X.t()).mm(X)/(M - torch.tensor(1))
                       
    e,v = torch.eig(C, eigenvectors=True) # v has column vectors
    e = e[:,0] # retrieve only the real parts
    
    # or use symeig instead since the matrix is symmetrical
    #e,v = torch.symeig(C, eigenvectors=True)
    #print(e,e.size())
    
    sorted, indices = torch.sort(e, dim=0, descending=True)
    v = v[:,indices].t()
    
    return mean,v
    
# Build the mean and A matrix of the dataset
mean,A = PCA(train_input.clone())
print('Mean and A sizes: ',mean.size(), A.size())

Mean and A sizes:  torch.Size([784]) torch.Size([784, 784])


In [80]:
'''
    Exo 4) Apply PCA to the data set and evaluate the k-NN nearest neighbour (for k = 1)
    Choose either 100 random eigenvectors or the first 3, 10, 50, 100 best eigenvectors.
'''
# Pick 100 random eigenvectors
r = torch.randint(0, train_input.size()[1], (100,))
Arand = A[r,:]

# Pick the first n eigenvectors
A3 = A[0:3,:]
A10 = A[0:10,:]
A50 = A[0:50,:]
A100 = A[0:100,:]

nb_errors_rand = compute_nb_errors(train_input.clone(), train_target.clone(), 
                        test_input.clone(), test_target.clone(),
                        mean = mean, proj = Arand)
print('Errors with Arand: ',nb_errors_rand)

nb_errors_rand = compute_nb_errors(train_input.clone(), train_target.clone(), 
                        test_input.clone(), test_target.clone(),
                        mean = mean, proj = A3)
print('Errors with A3: ',nb_errors_rand)

nb_errors_rand = compute_nb_errors(train_input.clone(), train_target.clone(), 
                        test_input.clone(), test_target.clone(),
                        mean = mean, proj = A10)
print('Errors with A10: ',nb_errors_rand)

nb_errors_rand = compute_nb_errors(train_input.clone(), train_target.clone(), 
                        test_input.clone(), test_target.clone(),
                        mean = mean, proj = A100)
print('Errors with A100: ',nb_errors_rand)

Errors with Arand:  585
Errors with A3:  597
Errors with A10:  214
Errors with A100:  164
